In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data = pd.read_csv('drive/My Drive/Humana/HMAHCC_COMP.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
 data.id.nunique()

14000

In [0]:
data.info()

In [0]:
data_me = data[(~data.MME.isnull())&(data.event_descr=='RX Claim - Paid')]

In [0]:
data_me['supply_count'] = pd.to_numeric(data_me.PAY_DAY_SUPPLY_CNT,errors='coerce')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
data_me['id_new'] = data_me.id

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
data_me['idx']=np.arange(data_me.shape[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
data_me.set_index(['idx','id'],inplace=True)

In [0]:
data_me.index[data_me.id_new=='ID1002482139'].to_list()

In [0]:
shifted = data_me.groupby(level="id").shift(-1)
# bring the next row data into current row


In [0]:
shifted_lead = data_me.groupby(level="id").shift(1)
# brings previous row data into current row

In [0]:
data_new = data_me[['id_new','Days','PAY_DAY_SUPPLY_CNT', 'PAYABLE_QTY', 'MME','supply_count']].join(shifted[['supply_count','Days']], rsuffix = "_lag")

In [0]:
data_new = data_new.join(shifted_lead[['supply_count','Days']], rsuffix = "_lead")

In [0]:
data_new.Days = pd.to_numeric(data_new.Days,errors='coerce')

In [0]:
data_new.Days_lag = pd.to_numeric(data_new.Days_lag,errors = 'coerce')

In [0]:
data_new.Days_lead = pd.to_numeric(data_new.Days_lead,errors='coerce')

In [0]:
data_new['diff_lag'] = data_new.Days_lag- data_new.Days

In [0]:
data_new['diff_lead'] = data_new.Days - data_new.Days_lead

In [0]:
data_new.head()

,,id_new,Days,PAY_DAY_SUPPLY_CNT,PAYABLE_QTY,MME,supply_count,supply_count_lag,Days_lag,supply_count_lead,Days_lead,diff_lag,diff_lead
idx,id,,,,,,,,,,,,
0,ID10010854159,ID10010854159,0,5.0,15.0,15.0,5.0,5.0,706.0,NaN,NaN,706.0,NaN
1,ID10010854159,ID10010854159,706,5.0,14.0,12.6,5.0,NaN,NaN,5.0,0.0,NaN,706.0
2,ID10013863216,ID10013863216,0,90.0,180.0,10.0,90.0,90.0,94.0,NaN,NaN,94.0,NaN
3,ID10013863216,ID10013863216,94,90.0,180.0,10.0,90.0,90.0,253.0,90.0,0.0,159.0,94.0
4,ID10013863216,ID10013863216,253,90.0,180.0,10.0,90.0,90.0,342.0,90.0,94.0,89.0,159.0


In [0]:
data_new[['diff_lead','supply_count_lead']].min(axis=1)

idx     id           
0       ID10010854159     NaN
1       ID10010854159     5.0
2       ID10013863216     NaN
3       ID10013863216    90.0
4       ID10013863216    90.0
5       ID10013863216    89.0
6       ID10013863216    90.0
7       ID10013863216    90.0
8       ID10013863216    30.0
9       ID10013863216    27.0
10      ID10013863216    30.0
11      ID10013863216    28.0
12      ID10013863216    30.0
13      ID10013863216    30.0
14      ID10024447278     NaN
15      ID10024447278     5.0
16      ID10024447278     5.0
17      ID10024447278     5.0
18      ID10024447278     3.0
19      ID10024447278    28.0
20      ID10024447278    30.0
21      ID10024447278    28.0
22      ID10024447278    28.0
23      ID10024447278    28.0
24      ID10024447278    29.0
25      ID10024447278    30.0
26      ID10024447278    30.0
27      ID10024447278    30.0
28      ID10024447278    30.0
29      ID10024447278    28.0
                         ... 
244492  ID99994197161    25.0
244493  ID99994197

In [0]:
data_new['naive'] = 0
data_new.loc[((data_new.diff_lead.isnull())|((data_new.diff_lead - 90)>data_new.supply_count_lead))&(data_new.Days>=0),'naive'] = 1

In [0]:
data_new.reset_index(inplace=True)

In [0]:
data_new.head()

,idx,id,id_new,Days,PAY_DAY_SUPPLY_CNT,PAYABLE_QTY,MME,supply_count,supply_count_lag,Days_lag,supply_count_lead,Days_lead,diff_lag,diff_lead,naive
0,0,ID10010854159,ID10010854159,0,5.0,15.0,15.0,5.0,5.0,706.0,NaN,NaN,706.0,NaN,1
1,1,ID10010854159,ID10010854159,706,5.0,14.0,12.6,5.0,NaN,NaN,5.0,0.0,NaN,706.0,1
2,2,ID10013863216,ID10013863216,0,90.0,180.0,10.0,90.0,90.0,94.0,NaN,NaN,94.0,NaN,1
3,3,ID10013863216,ID10013863216,94,90.0,180.0,10.0,90.0,90.0,253.0,90.0,0.0,159.0,94.0,0
4,4,ID10013863216,ID10013863216,253,90.0,180.0,10.0,90.0,90.0,342.0,90.0,94.0,89.0,159.0,0


In [0]:
target_df = pd.DataFrame(columns=['ID','Target'])
index=0
c=0
for i in data_new.id_new.unique():
  # iterating for each unique user id
  c+=1
  sub_set = data_new[data_new.id_new==i]
  sub_set.reset_index(inplace=True)
  for k in sub_set.index[sub_set.naive==1].to_list():
    # iterating for each opioid naive case for the respective user id
    ind = k
    sub_set_2 = sub_set[ind:]
    sub_set_2.reset_index(inplace=True)
    sub_set_2.Days_new = sub_set_2.Days-sub_set_2.Days[0]
    days=0
    supply_days = 0

    if len(sub_set_2)>=1:
      for j in range(sub_set_2.shape[0]):
        # calculating the no of days they have taken opiod medicine after each incident for each individual
        days+=sub_set_2.Days_new[j]
        supply_days = min(sub_set_2.Days_new[j],supply_days)
        supply_days+= sub_set_2.supply_count[j]
        if sub_set_2.Days_new[j]+sub_set_2.supply_count[j]>=162:
          remain_days = np.max([180-sub_set_2.Days[j],0])
          add_days = np.min([remain_days,sub_set_2.supply_count[j]])
          supply_days = supply_days-sub_set_2.supply_count[j]+add_days          
          if supply_days>=162:
            # if total supply >= 162 within 180 days then classify as one
            # print(supply_days)
            target_df = target_df.append(pd.DataFrame([(i,1)],columns=['ID','Target']))
          else:
            target_df = target_df.append(pd.DataFrame([(i,0)],columns=['ID','Target']))
          index+=1
          break
        # elif sub_set_2.Days_new[j]+sub_set_2.supply_count[j]>=162:
        # else:
        #   target_df = target_df.append(pd.DataFrame([(i,0)],columns=['ID','Target']))
          

    # else:
    #   # print((sub_set_2.supply_count >= 162))
    #   if (sub_set_2.supply_count >= 162).all():
    #     target_df.append(pd.DataFrame([i,1]))
    #   else:
    #     target_df.append(pd.DataFrame([i,1]))
    #   index+=1
    #     # print(sub_set_2.Days[j]-100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [0]:
type(min(sub_set_2.Days[1],supply_days))

numpy.float64

In [0]:
data_new[data_new.naive==1].shape

(26810, 13)

In [0]:
target_var = target_df.groupby('ID').max()
target_var.reset_index(inplace=True)

In [0]:
cd drive/My\ Drive/Humana

In [0]:
ls

'call category.csv'      disposition.csv   HMAHCC_HOLDOUT.csv
'call category.gsheet'   HMAHCC_COMP.csv   target_var1_final.csv


In [0]:
target_var.to_csv('target_var1_final.csv')

In [0]:
target_df.Target.value_counts()

0    11092
1     4481
Name: Target, dtype: int64

In [0]:
target_var[target_var.Target==1]['ID']

1        ID10013863216
2        ID10024447278
3         ID1002482139
8        ID10112532931
10       ID10125698107
14       ID10166893764
15       ID10167700196
20       ID10203805803
21       ID10212367358
22       ID10230809175
23       ID10234639565
25       ID10249588270
26       ID10256258775
30       ID10309381787
34       ID10362163889
35       ID10363405901
36       ID10366078610
37       ID10395772108
41       ID10409931331
46       ID10444101509
56       ID10551807382
59       ID10572785190
61         ID105838524
63       ID10597480000
64       ID10607745549
68       ID10665075020
69       ID10680847713
70       ID10681401244
71       ID10682483068
72        ID1070574904
             ...      
10303    ID99272306821
10304    ID99307164642
10306    ID99323519412
10309    ID99352641962
10310    ID99357693828
10311     ID9936081436
10312    ID99367966828
10314    ID99399800505
10316    ID99421949451
10319    ID99455307516
10322    ID99491178290
10326    ID99527660663
10330    ID

In [0]:
count_table = data[['id','event_descr']].groupby(['id','event_descr']).size().reset_index(name='count')

In [0]:
df = count_table.set_index(['id','event_descr'])['count'].unstack()

In [0]:
df.reset_index(inplace=True)

In [0]:
df.fillna(0,inplace=True)

In [0]:
df.head()

In [0]:
df.to_csv('independent_var.csv',index=False)

In [0]:
count_table.to_csv('count_table.csv')

In [0]:
data_diag = data.loc[data.event_descr.isin(['New diagnosis - CAD',
       'New diagnosis - CPD', 'New diagnosis - CHF','Surgery',
       'New diagnosis - Hypertension', 'New diagnosis - Top 5',
       'New diagnosis - Diabetes']),:]

In [0]:
data_diag.event_attr1.unique()

array(['OTHER AND COMBINED FORMS OF SENILE CATARACT',
       'DIZZINESS AND GIDDINESS',
       'OTHER SPECIFIED NONINFLAMMATORY DISORDERS OF VAGINA', ...,
       'POSTCONCUSSIONAL SYNDROME', 'OTHER SPECIFIED DISORDERS OF MUSCLE',
       'BURN OF SECOND DEGREE OF SHOULDER AND UPPER LIMB, EXCEPT WRIST AND HAND, UNSPECIFIED SITE, SUBSEQUENT ENCOUNTER'],
      dtype=object)

In [0]:
data_drug = data[(data.MME.isnull())&(data.event_descr=='RX Claim - Paid')]

In [0]:
data_drug.head()

,id,event_descr,event_attr1,event_attr2,event_attr3,event_attr4,event_attr5,event_attr6,event_attr7,event_attr8,event_attr9,event_attr10,Days,PAY_DAY_SUPPLY_CNT,PAYABLE_QTY,MME,DRUG_TYPE,Specialty,Specialty2,Specialty3
0,ID10010854159,RX Claim - Paid,BIGUANIDES,NaN,5,2.35,METFORMIN HCL,DIABETES,NaN,METFORMIN HCL TAB 500 MG,2.65,27250050.0,-664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID10010854159,RX Claim - Paid,BETA BLOCKERS CARDIO-SELECTIVE,NaN,5,2.35,ATENOLOL,CARDIO,NaN,ATENOLOL TAB 25 MG,2.65,33200020.0,-659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID10010854159,RX Claim - Paid,FLUOROQUINOLONES,NaN,5.8,3.15,CIPROFLOXACIN HCL,INF-ANTIBIOTICS,NaN,CIPROFLOXACIN HCL TAB 500 MG (BASE EQUIV),2.65,5000020.0,-646,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID10010854159,RX Claim - Paid,HMG COA REDUCTASE INHIBITORS,NaN,13,10.35,PRAVASTATIN SODIUM,CARDIO,NaN,PRAVASTATIN SODIUM TAB 20 MG,2.65,39400065.0,-646,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ID10010854159,RX Claim - Paid,THIAZIDES AND THIAZIDE-LIKE DIURETICS,NaN,5,2.35,HYDROCHLOROTHIAZIDE,CARDIO,NaN,HYDROCHLOROTHIAZIDE TAB 25 MG,2.65,37600040.0,-644,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [0]:
data_drug['pain'] = 0
data_drug.loc[data_drug.event_attr6.isin(['PAIN']),'pain']=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [0]:
pain_count = data_drug[['id','pain']].groupby(['id','pain']).size().reset_index(name='count').set_index(['id','pain'])['count'].unstack()

In [0]:
pain_count.reset_index(inplace=True)

In [0]:
pain_count.fillna(0,inplace=True)
# [2] = 'Pain'
#  = ['id','Non_pain','pain']

In [0]:
pain_count.to_csv('pain_count.csv',index=False)

In [0]:
data_opiod = data_drug[data_drug.event_attr6.isin(['PAIN'])]

In [0]:
data_opiod[data_opiod.event_attr1.str.contains('OPIOID')].head()

In [0]:
data_opiod['OPIOID'] = 'Non Opioid'
data_opiod.loc[data_opiod.event_attr1.str.contains('OPIOID'),'OPIOID'] = 'Opioid'

In [0]:
opioid_count = data_opiod[['id','OPIOID']].groupby(['id','OPIOID']).size().reset_index(name='count').\
set_index(['id','OPIOID'])['count'].unstack()
opioid_count.reset_index(inplace=True)
opioid_count.fillna(0,inplace=True)
opioid_count.to_csv('opioid_count.csv',index=False)

In [0]:
data_me.Days.min()

-1270

In [0]:
# This is not required
# data_drug['OPIOID'] = 'Non Opioid'
# data_drug.loc[data_drug.event_attr1.str.contains('OPIOID').fillna(False),'OPIOID'] = 'Opioid'
# opioid_count_drug = data_drug[['id','OPIOID']].groupby(['id','OPIOID']).size().reset_index(name='count').\
# set_index(['id','OPIOID'])['count'].unstack()
# opioid_count_drug.reset_index(inplace=True)
# opioid_count_drug.fillna(0,inplace=True)
# opioid_count_drug.to_csv('opioid_count_drug.csv',index=False)

In [0]:
for event_desc in data.event_descr.unique():
  print(event_desc + ': ' + str(data.event_attr1[data.event_descr.isin([event_desc])].nunique()))

RX Claim - Paid: 437
RX Claim - Rejected: 8
Fully Paid Claim: 17500
RX Claim - New Drug: 643
Inbound Call by Mbr: 342
Inbound Call by Prov: 107
Surgery: 3514
RX Claim - First Time Mail Order: 187
New provider: 0
Inbound Call by Other: 151
New diagnosis - CAD: 744
New diagnosis - CPD: 969
New diagnosis - CHF: 600
New diagnosis - Hypertension: 782
New diagnosis - Top 5: 772
New diagnosis - Diabetes: 672


In [0]:
holdout = pd.read_csv('drive/My Drive/Humana/HMAHCC_HOLDOUT.csv',dtype='unicode')

In [0]:
holdout.shape

(1480394, 20)

In [0]:
holdout.ID.unique()

array(['ID98975196852', 'ID40721836565', 'ID99208242445', ...,
       'ID98960792971', 'ID78706797900', 'ID10278406324'], dtype=object)

In [0]:
holdout[holdout.ID=='ID98975196852'].to_csv('holdout_sample.csv')

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6086969 entries, 0 to 6086968
Data columns (total 20 columns):
id                    object
event_descr           object
event_attr1           object
event_attr2           object
event_attr3           object
event_attr4           object
event_attr5           object
event_attr6           object
event_attr7           object
event_attr8           object
event_attr9           object
event_attr10          object
Days                  object
PAY_DAY_SUPPLY_CNT    object
PAYABLE_QTY           object
MME                   object
DRUG_TYPE             object
Specialty             object
Specialty2            object
Specialty3            object
dtypes: object(20)
memory usage: 928.8+ MB
